# 使用SVM进行图片分类预测

In [1]:
%matplotlib inline

from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVC
from sklearn import metrics, datasets
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split

from skimage.io import imread
from skimage.transform import resize

import cv2
import numpy as np
from sklearn.svm import SVC
import joblib
from skimage.io import imread
from skimage.transform import resize
from sklearn.preprocessing import StandardScaler
import os

### 读取本地图像数据

In [2]:
def load_image_files(container_path, dimension=(64, 64)):
    """
    读取不同类别的图像数据
    并将其转换成sklearn能够处理的数据集
    
    参数说明
    ----------
    container_path : 存放图片数据集的本地路径
    dimension : 调整图片的尺寸

    """
    # 获取存储图像的文件夹位置
    image_dir = Path(container_path)
    # 存储图像的文件夹位置
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    # 根据存放不同类别图像的文件夹名，获取图像的类别
    categories = [fo.name for fo in folders]

    descr = "A image classification dataset"
    images = [] # 用于存储所有图像数组的列表变量
    flat_data = [] # 用于存储展平后的图像数据的列表变量
    target = [] # 用于存储所有标签的列表变量

    # 遍历每个类别的文件夹，读取其中的所有图像
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            img = imread(file) # 读取图像
            img_resized = resize(img, dimension, anti_aliasing=True, mode='reflect') # 裁剪调整图像尺寸
            flat_data.append(img_resized.flatten()) # 将展平后的图像数据添加到flat_data中
            images.append(img_resized) # 将原始图像数组添加到images中
            target.append(i) # 将该图像对应的标签添加到target中

    flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)

    # 将所有的图像和相关信息保存到Bunch容器中
    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 images=images,
                 DESCR=descr)


In [3]:
# image_dataset = load_image_files("images/")
# image_dataset = load_image_files("data/train")
# image_dataset

In [4]:
import pickle
# Save the preprocessed dataset as a pickle file
# with open('preprocessed_dataset.pickle', 'wb') as f:
#     pickle.dump(image_dataset, f)

# 从pickle文件中读取数据
with open('preprocessed_dataset.pickle', 'rb') as f:
    image_dataset = pickle.load(f)

In [5]:
image_dataset

{'data': array([[0.95648765, 0.79962367, 0.66237491, ..., 0.44689823, 0.34886024,
         0.26258328],
        [0.82289657, 0.72093333, 0.63073848, ..., 0.66863733, 0.56667654,
         0.47647801],
        [0.96252769, 0.95076299, 0.93115514, ..., 0.8216962 , 0.8216962 ,
         0.8216962 ],
        ...,
        [0.88245772, 0.81579106, 0.74520282, ..., 0.35239109, 0.31709698,
         0.25827345],
        [0.84750589, 0.7965255 , 0.72201569, ..., 0.34118297, 0.30588885,
         0.24706532],
        [0.82791725, 0.75733024, 0.70242582, ..., 0.47114278, 0.4319271 ,
         0.39270896]]),
 'target': array([0, 0, 0, ..., 5, 5, 5]),
 'target_names': ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash'],
 'images': array([[[[0.95648765, 0.79962367, 0.66237491],
          [0.95309376, 0.79572844, 0.66098638],
          [0.9535326 , 0.79324717, 0.6730885 ],
          ...,
          [0.82656383, 0.65401481, 0.54028932],
          [0.84687789, 0.67432887, 0.56060338],
          [0.8

In [6]:
dir(image_dataset)

['DESCR', 'data', 'images', 'target', 'target_names']

In [7]:
image_dataset.data.shape

(2407, 12288)

In [8]:
type(image_dataset)

sklearn.utils.Bunch

### 数据集划分
按照8:2的比例划分训练集和测试集

In [9]:
# 导入train_test_split方法
from sklearn.model_selection import train_test_split

# 使用train_test_split方法将图像数据集划分为训练集和测试集
# image_dataset.data包含所有图像数据，image_dataset.target包含每张图像对应的标签
# test_size=0.2表示测试集占总数据集的20%
# random_state=109表示随机种子，确保每次运行结果相同
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset.data,  # 所有图像数据
    image_dataset.target,  # 每张图像对应的标签
    test_size=0.2,  # 测试集占总数据集的20%
    random_state=109)  # 随机种子，确保每次运行结果相同

In [10]:
# 导入SVM模型
from sklearn.svm import SVC

# 实例化SVM模型，kernel='rbf'表示使用RBF核函数，C=10表示惩罚系数为10，gamma=0.001表示RBF核函数的带宽参数为0.001
svc = SVC(kernel='rbf', C=10, gamma=0.001)

# 使用训练集数据X_train和y_train来训练SVM模型
svc.fit(X_train, y_train)


# Load the model from disk
# loaded_model = joblib.load('model/model_file_kernel_linear_C10.sav')

SVC(C=10, gamma=0.001)

In [11]:
# 使用已经训练好的 SVM 模型对测试数据集进行预测
yfit = svc.predict(X_test)

# 比较预测结果和真实标签，并输出分类报告
print(metrics.classification_report(y_test, yfit))

              precision    recall  f1-score   support

           0       0.36      0.58      0.44        73
           1       0.33      0.35      0.34        96
           2       0.31      0.24      0.27        76
           3       0.67      0.53      0.59       116
           4       0.45      0.40      0.43        97
           5       0.33      0.38      0.35        24

    accuracy                           0.42       482
   macro avg       0.41      0.41      0.40       482
weighted avg       0.44      0.42      0.42       482



In [18]:
import joblib

joblib.dump(svc, 'model/model_file_kernel_linear_C10.sav')

['model/model_file_kernel_linear_C10.sav']

In [7]:
#使用线性核函数（linear），去掉gamma参数，并绘制不同C参数下的训练精度折线图：
import matplotlib.pyplot as plt
from sklearn import svm

import time

# 开始计时
start_time = time.time()

# 线性核函数
kernel = 'linear'

# 参数范围
C_range = [0.1, 1, 10, 100]

# 创建一个新图形
fig = plt.figure()

# 循环遍历不同的参数组合并绘制折线图
for i, C in enumerate(C_range):
    # 训练精度列表
    training_accuracies = []
    
    # 创建SVM分类器，并训练模型
    clf = svm.SVC(kernel=kernel, C=C)
    clf.fit(X_train, y_train)

    # 计算训练精度
    accuracy = clf.score(X_train, y_train)
    training_accuracies.append(accuracy)
    
    # 绘制折线图
    plt.plot([C], training_accuracies, '-o', label='C: {}'.format(C))

# 添加图例、标题和标签
plt.legend()
plt.title('Training Accuracies by C for Linear Kernel')
plt.xlabel('C Parameter')
plt.ylabel('Training Accuracy')

# 显示图形
plt.show()


# 结束计时
end_time = time.time()

# 输出时间消耗
print('Training time: {:.2f} seconds'.format(end_time - start_time))


In [ ]:
import matplotlib.pyplot as plt
from sklearn import svm

import time

# 开始计时
start_time = time.time()

# RBF核函数
kernel = 'rbf'

# 参数范围
C_range = [0.1, 1, 10, 100]
gamma_range = [0.01, 0.1, 1, 10]

# 创建一个新图形
fig = plt.figure()

# 循环遍历不同的参数组合并绘制折线图
for i, gamma in enumerate(gamma_range):
    # 训练精度列表
    training_accuracies = []
    
    for C in C_range:
        # 创建SVM分类器，并训练模型
        clf = svm.SVC(kernel=kernel, C=C, gamma=gamma)
        clf.fit(X_train, y_train)
        
        # 计算训练精度
        accuracy = clf.score(X_train, y_train)
        training_accuracies.append(accuracy)
    
    # 绘制折线图
    plt.plot(C_range, training_accuracies, '-o', label='Gamma: {}'.format(gamma))

# 添加图例、标题和标签
plt.legend()
plt.title('Training Accuracies by C and Gamma for RBF Kernel')
plt.xlabel('C Parameter')
plt.ylabel('Training Accuracy')

# 显示图形
plt.show()


# 结束计时
end_time = time.time()

# 输出时间消耗
print('Training time: {:.2f} seconds'.format(end_time - start_time))

使用网格搜索方法进行模型参数调优


#### 设置参数的搜索空间

In [19]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

# 模型参数空间
param_grid = [
    {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
    {'C': [1, 10, 100, 1000], 'degree':[2,3,4], 'gamma': ['auto'],'kernel': ['poly']}
]

# 创建一个SVC模型，并使用GridSearchCV和5折交叉验证来选择最佳超参数
grid = GridSearchCV(SVC(class_weight='balanced'), param_grid,cv=5)

# 训练模型、选择最佳超参数
%time grid.fit(X_train, y_train)

# 获取最佳模型
model = grid.best_estimator_

# 在测试集上进行模型预测
y_pred = model.predict(X_test)

# 查看模型预测结果
print("Classification report for - \n{}:\n{}\n".format(model, metrics.classification_report(y_test, y_pred)))

In [ ]:
grid.best_estimator_

SVC(C=10, class_weight='balanced', gamma=0.001)

In [ ]:
grid.best_estimator_.kernel

'rbf'

In [ ]:
model = grid.best_estimator_

### 在测试集上进行模型预测

In [ ]:
y_pred = model.predict(X_test)

### 查看模型预测结果

In [ ]:
print("Classification report for - \n{}:\n{}\n".format(model, metrics.classification_report(y_test, y_pred)))

Classification report for - 
SVC(C=10, class_weight='balanced', gamma=0.001):
              precision    recall  f1-score   support

           0       0.75      0.55      0.63        22
           1       0.87      0.76      0.81        17
           2       0.85      0.79      0.81        14
           3       0.55      0.71      0.62        17
           4       0.85      1.00      0.92        23

    accuracy                           0.76        93
   macro avg       0.77      0.76      0.76        93
weighted avg       0.77      0.76      0.76        93




In [17]:
import cv2
import numpy as np
from sklearn.svm import SVC
import joblib
from skimage.io import imread
from skimage.transform import resize
from sklearn.preprocessing import StandardScaler


# 加载已经训练好的SVM模型
# model = SVC(kernel='linear', C=10, probability=True, random_state=42)
# Load the model from disk
loaded_model = joblib.load('model/model_file_kernel_linear_C10.sav')

# 读取待分类的图像
img = imread('data/validation/glass/glass485.jpg')

# 预处理图像数据
# 调整图像尺寸
img_resized = resize(img, (64, 64), anti_aliasing=True, mode='reflect')

# 将图像展平为一维数组结构
flat_data = img_resized.flatten()

# 使用训练集中使用过的标准化方法，对测试数据进行标准化处理
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform([flat_data])

# 使用训练好的SVM模型进行分类预测
y_pred = loaded_model.predict(X_test_scaled)


# 获取预测结果的标签
labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
predicted_label = labels[int(y_pred)]

#在图片上显示预测结果
img = cv2.putText(img=img, text=predicted_label, org=(50, 50), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.5, color=(0, 0, 255), thickness=2, lineType=cv2.LINE_AA)

# 展示预测结果
cv2.imshow('Test Image', img)
print("The predicted label for this image is: ", predicted_label)

cv2.waitKey(0)
cv2.destroyAllWindows()


The predicted label for this image is:  glass


In [24]:
# 加载已经训练好的SVM模型
loaded_model = joblib.load('model/the_best_model_file_kernel_rbf_C10_gamma0001.sav')

# 读取待分类的图像所在的文件夹路径
folder_path = 'data/validation-2/'

# 获取文件夹中所有图片的路径
images_list = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg')]

# 初始化当前显示的图片索引
current_idx = 0

while True:
    # 读取待分类的图像
    img = imread(images_list[current_idx])

    # 预处理图像数据
    # 调整图像尺寸
    img_resized = resize(img, (64, 64), anti_aliasing=True, mode='reflect')

    # 将图像展平为一维数组结构
    flat_data = img_resized.flatten()

    # 使用训练集中使用过的标准化方法，对测试数据进行标准化处理
    scaler = StandardScaler()
    X_test_scaled = scaler.fit_transform([flat_data])

    # 使用训练好的SVM模型进行分类预测
    y_pred = loaded_model.predict(X_test_scaled)

    # 获取预测结果的标签
    labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
    labels_2 = ['cardboard', 'cardboard','cardboard','glass', 'glass',
                'glass','glass','metal', 'paper', 'paper', 'trash']
    predicted_label = labels[int(y_pred)]

    # 在图片上显示预测结果
    # img = cv2.putText(img=img, text=predicted_label, org=(50, 50), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.5, color=(0, 0, 255), thickness=2, lineType=cv2.LINE_AA)

    # 展示预测结果
    cv2.imshow('Test Image', img)
    print("The predicted label for this image is: ", predicted_label)

    # 等待用户按键输入
    key = cv2.waitKey(0)

    # 如果按下左箭头，切换到前一张图片
    if key == ord('a') and current_idx > 0:
        # 关闭之前的图片
        cv2.destroyAllWindows()
        current_idx -= 1
        # 重新读取当前索引对应的图片并进行分类预测
        img = imread(images_list[current_idx])
        img_resized = resize(img, (64, 64), anti_aliasing=True, mode='reflect')
        flat_data = img_resized.flatten()
        X_test_scaled = scaler.transform([flat_data])
        y_pred = loaded_model.predict(X_test_scaled)
        predicted_label = labels[int(y_pred)]
        img = cv2.putText(img=img, text=predicted_label, org=(50, 50), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.5, color=(0, 0, 255), thickness=2, lineType=cv2.LINE_AA)
        cv2.imshow('Left Image', img)
        print("The predicted label for this image is: ", predicted_label)

    # 如果按下右箭头，切换到后一张图片
    elif key == ord('d') and current_idx < len(images_list) - 1:
    # elif key == ord('d'):
        # 关闭之前的图片
        cv2.destroyAllWindows()
        current_idx += 1
        # 重新读取当前索引对应的图片并进行分类预测
        img = imread(images_list[current_idx])
        img_resized = resize(img, (64, 64), anti_aliasing=True, mode='reflect')
        flat_data = img_resized.flatten()
        X_test_scaled = scaler.transform([flat_data])
        y_pred = loaded_model.predict(X_test_scaled)
        predicted_label = labels[int(y_pred)]
        img = cv2.putText(img=img, text=predicted_label, org=(50, 50), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.5, color=(0, 0, 255), thickness=2, lineType=cv2.LINE_AA)
        # img = cv2.putText(img=img, text=labels_2[int(current_idx)], org=(50, 50), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.5, color=(0, 0, 255), thickness=2, lineType=cv2.LINE_AA)
        cv2.imshow('Right Image', img)
        print("The predicted label for this image is: ", predicted_label)

    # 如果按下ESC键，退出程序
    elif key== 27:
        break

cv2.destroyAllWindows()

The predicted label for this image is:  paper
The predicted label for this image is:  paper
The predicted label for this image is:  paper
The predicted label for this image is:  paper
The predicted label for this image is:  paper
The predicted label for this image is:  paper
The predicted label for this image is:  paper
The predicted label for this image is:  paper
The predicted label for this image is:  paper
The predicted label for this image is:  paper
The predicted label for this image is:  paper
The predicted label for this image is:  metal
The predicted label for this image is:  paper
The predicted label for this image is:  paper
The predicted label for this image is:  paper
The predicted label for this image is:  paper
The predicted label for this image is:  paper
The predicted label for this image is:  metal
The predicted label for this image is:  paper


In [ ]:
# 导入需要的模块
import os
import cv2
from skimage.io import imread
from skimage.transform import resize
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib

# 加载已经训练好的SVM模型
loaded_model = joblib.load('model/the_best_model_file_kernel_rbf_C10_gamma0001.sav')

# 读取待分类的图像所在的文件夹路径
folder_path = 'data/validation-2/'

# 获取文件夹中所有图片的路径
images_list = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg')]

# 初始化当前显示的图片索引
current_idx = 0

while True:
    # 读取待分类的图像
    img = imread(images_list[current_idx])

    # 预处理图像数据
    # 调整图像尺寸
    img_resized = resize(img, (64, 64), anti_aliasing=True, mode='reflect')

    # 将图像展平为一维数组结构
    flat_data = img_resized.flatten()

    # 使用训练集中使用过的标准化方法，对测试数据进行标准化处理
    scaler = StandardScaler()
    X_test_scaled = scaler.fit_transform([flat_data])

    # 使用训练好的SVM模型进行分类预测
    y_pred = loaded_model.predict(X_test_scaled)

    # 获取预测结果的标签
    labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
    predicted_label = labels[int(y_pred)]

    # 在图片上显示预测结果
    img = cv2.putText(img=img, text=predicted_label, org=(50, 50), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.5, color=(0, 0, 255), thickness=2, lineType=cv2.LINE_AA)

    # 展示预测结果
    cv2.imshow('Test Image', img)
    print("The predicted label for this image is: ", predicted_label)

    # 等待用户按键输入
    key = cv2.waitKey(0)

    # 如果按下左箭头，切换到前一张图片
    if key == ord('a') and current_idx > 0:
        # 关闭之前的图片
        cv2.destroyAllWindows()
        current_idx -= 1
        # 重新读取当前索引对应的图片并进行分类预测
        img = imread(images_list[current_idx])
        img_resized = resize(img, (64, 64), anti_aliasing=True, mode='reflect')
        flat_data = img_resized.flatten()
        X_test_scaled = scaler.transform([flat_data])
        y_pred = loaded_model.predict(X_test_scaled)
        predicted_label = labels[int(y_pred)]
        img = cv2.putText(img=img, text=predicted_label, org=(50, 50), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.5, color=(0, 0, 255), thickness=2, lineType=cv2.LINE_AA)
        cv2.imshow('Left Image', img)
        print("The predicted label for this image is: ", predicted_label)

    # 如果按下右箭头，切换到后一张图片
    elif key == ord('d') and current_idx < len(images_list) - 1:
        # 关闭之前的图片
        cv2.destroyAllWindows()
        current_idx += 1
        # 重新读取当前索引对应的图片并进行分类预测
        img = imread(images_list[current_idx])
        img_resized = resize(img, (64, 64), anti_aliasing=True, mode='reflect')
        flat_data = img_resized.flatten()
        X_test_scaled = scaler.transform([flat_data])
        y_pred = loaded_model.predict(X_test_scaled)
        predicted_label = labels[int(y_pred)]
        img = cv2.putText(img=img, text=predicted_label, org=(50, 50), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.5, color=(0, 0, 255), thickness=2, lineType=cv2.LINE_AA)
        cv2.imshow('Right Image', img)
        print("The predicted label for this image is: ", predicted_label)

    # 如果按下ESC键，退出程序
    elif key== 27:
        break

# 关闭所有窗口
cv2.destroyAllWindows()